In [1]:
# from IPython.display import clear_output
# import openml # openML
import pandas as pd
import numpy as np
from os import listdir # get dir list
from os.path import isfile, join # get dir list
import random
# from pymfe.mfe import MFE

In [2]:
def getFolderFilesList(path):
    return [filename for filename in listdir(path) if isfile(join(path, filename))]

In [3]:
dictOfFIles = {'DT': [], 'RF': [], 'KNN': [], 'SVM': [] }
filtersList = getFolderFilesList('Results')
for i in filtersList:
    if (i.split('_')[-1] == 'resultsDT.csv.zip'):
        dictOfFIles['DT'].append(i)
    elif (i.split('_')[-1] == 'resultsRF.csv.zip'):
        dictOfFIles['RF'].append(i)
    elif (i.split('_')[-1] == 'resultsSVM.csv.zip'):
        dictOfFIles['SVM'].append(i)
    else:
        dictOfFIles['KNN'].append(i)

metaFeatures = pd.read_csv('meta_features_dropna.csv')
metaFeatures = metaFeatures.set_index('Dataset').sort_index()
# metaFeatures = metaFeatures.drop(['lh_trace', 'roy_root'], axis = 1)
metabase = {'DT': metaFeatures.copy(), 'RF': metaFeatures.copy(), 'KNN': metaFeatures.copy(), 'SVM': metaFeatures.copy() }
for value in dictOfFIles:
    for file in dictOfFIles[value]:
        df = pd.read_csv(f'Results/{file}')
        filterName = file.split('_')[1]
        df = df.rename({'Unnamed: 0': 'Dataset', 'test_f1_macro_mean': f'test_f1_macro_mean_{filterName}'},  axis=1)
        df = df.set_index('Dataset')
        df = df.sort_index()
        metabase[value] = metabase[value].join(df[f'test_f1_macro_mean_{filterName}'])

In [6]:
metabase['SVM'].iloc[:, 97:]

,test_f1_macro_mean_ORBoost-15,test_f1_macro_mean_dynamicCF,test_f1_macro_mean_ORBoost-7,test_f1_macro_mean_edgeBoostFilter-15,test_f1_macro_mean_edgeBoostFilter-20,test_f1_macro_mean_AENN-3,test_f1_macro_mean_GE-5,test_f1_macro_mean_GE-9,test_f1_macro_mean_HARF-70,test_f1_macro_mean_GE-7,...,test_f1_macro_mean_AENN-5,test_f1_macro_mean_ORBoost-3,test_f1_macro_mean_HARF-80,test_f1_macro_mean_HARF-75,test_f1_macro_mean_ORBoost-19,test_f1_macro_mean_PRISM,test_f1_macro_mean_edgeBoostFilter-5,test_f1_macro_mean_HARF-85,test_f1_macro_mean_hybridRepairFilter,test_f1_macro_mean_HARF-90
Dataset,,,,,,,,,,,,,,,,,,,,,
1004_synthetic_control_10_0.csv.gz,1.000000,0.996590,1.000000,0.919346,0.960037,0.978341,0.967043,1.000000,0.986979,0.993877,...,0.978341,1.000000,0.971459,0.986979,1.000000,0.952636,0.835194,0.944923,1.000000,0.857858
1004_synthetic_control_10_1.csv.gz,1.000000,1.000000,1.000000,0.827969,0.992051,0.966712,0.985043,1.000000,0.993346,0.997058,...,0.966712,1.000000,0.961939,0.978043,1.000000,0.960775,0.827297,0.946303,0.990938,0.874056
1004_synthetic_control_10_2.csv.gz,1.000000,1.000000,1.000000,0.874679,0.984000,0.973128,0.977559,1.000000,1.000000,0.990805,...,0.973128,1.000000,0.970258,1.000000,1.000000,0.956329,0.764287,0.935619,1.000000,0.866705
1004_synthetic_control_10_3.csv.gz,1.000000,1.000000,1.000000,0.951843,0.951626,0.969475,0.990815,0.994000,0.989764,0.987493,...,0.969475,1.000000,0.968975,0.983130,1.000000,0.966047,0.906751,0.933944,1.000000,0.890920
1004_synthetic_control_10_4.csv.gz,1.000000,1.000000,1.000000,0.983090,0.982119,0.996529,0.981190,1.000000,0.993333,1.000000,...,0.996529,1.000000,0.969977,0.989771,1.000000,0.980885,0.706128,0.929375,1.000000,0.833560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997_balance-scale_5_5.csv.gz,0.880494,0.873525,0.887928,0.798109,0.839966,0.890702,0.792649,0.809740,0.837700,0.810082,...,0.911735,0.887928,0.809447,0.816739,0.880494,0.834155,0.775168,0.815028,0.829695,0.800776
997_balance-scale_5_6.csv.gz,0.870356,0.886910,0.904376,0.774994,0.844899,0.888641,0.772242,0.771610,0.833213,0.772219,...,0.909238,0.890426,0.805183,0.805674,0.870356,0.812164,0.746827,0.788689,0.794555,0.775232
997_balance-scale_5_7.csv.gz,0.873039,0.880728,0.873966,0.815380,0.820393,0.868198,0.764739,0.825563,0.827520,0.811599,...,0.885852,0.874457,0.795973,0.808124,0.873039,0.800605,0.769124,0.799577,0.800859,0.798217


In [14]:
for value in metabase:
    for i in metabase[value].columns:
        if (metabase[value][i].isin([np.inf, -np.inf]).sum() > 0):
            metabase[value].drop(i, axis=1, inplace=True)
    metabase[value].fillna(0, inplace=True)
    metabase[value].to_csv(f'Metabases/Metabase_{value}.csv.zip')